In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import json

In [4]:
df = pd.read_excel("Input.xlsx")
df.head()

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL_ID  114 non-null    int64 
 1   URL     114 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [71]:
from nltk.corpus import stopwords
from string import punctuation
nltk.download("stopwords")
stopwords_word = stopwords.words("english")
punctuation_word = punctuation

[nltk_data] Downloading package stopwords to C:\Users\HP-
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:

df

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [ ]:
for url in df['URL']:
    print(url)
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    r = session.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    if (len(soup.find_all("body", class_="error404")) == 0):
        article = soup.find("article")
        article_title = article.find("h1")
        article_text = article.find_all("p")
        para = []
        for paragraph in article_text:
            para.append(paragraph.text)
        user_id = df[df["URL"] == url]["URL_ID"]
        obj = open(str(user_id.values[0])+".txt", 'w', errors="ignore")
        obj.write(article_title.string+"\n")
        obj.writelines(para)
    else:
        user_id = df[df["URL"] == url]["URL_ID"]
        obj = open(str(user_id.values[0])+".txt", 'w', errors="ignore")
        obj.write("404error\n")

    # print(para[0])

In [46]:
# text analysis
# sentiment analysis
import nltk
from nltk.tokenize import word_tokenize

In [125]:
def count_syllables(word):
    vovel="AEIOUaeiou"
    vovel_count=0
    complex=-1
    for A in word:
        if A in vovel:
            vovel_count+=1
    if(word[-1:-3:-1]=="se" or word[-1:-3:-1]=="de"):
        vovel_count-=1
    if(vovel_count>2):
        complex=1
    return(vovel_count,complex)



In [63]:
import re

def count_personal_pronouns(text):
  """Counts the number of personal pronouns in a text.

  Args:
    text: The text to be analyzed.

  Returns:
    The number of personal pronouns in the text.
  """

  pronouns = re.findall(r"(I|we|my|ours|us)[^US]", text)
  return len(pronouns)



In [134]:
outputfile = pd.read_excel('Output Data Structure.xlsx')
outputfile.head()

,Unnamed: 0.1,Unnamed: 0,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,0,0,37,https://insights.blackcoffer.com/ai-in-healthc...,63,31,0.340426,0.090559,36.849057,0.562620,21.425048,36.849057,584,1167,3.614644,94,5.282130
1,1,1,38,https://insights.blackcoffer.com/what-if-the-c...,55,36,0.208791,0.134417,23.808824,0.432792,27.373117,23.808824,293,809,3.748892,68,4.346510
2,2,2,39,https://insights.blackcoffer.com/what-jobs-wil...,65,34,0.313131,0.107609,27.500000,0.554348,27.421739,27.500000,510,1032,3.722826,87,5.001070
3,3,3,40,https://insights.blackcoffer.com/will-machine-...,55,21,0.447368,0.103121,22.855263,0.468114,30.587246,22.855263,345,924,3.986431,120,4.516983
4,4,4,41,https://insights.blackcoffer.com/will-ai-repla...,47,22,0.362319,0.076582,28.552239,0.437292,26.974917,28.552239,394,1028,3.507214,93,4.626242


In [135]:
for file in df["URL_ID"]:
    obj = open(str(file)+".txt", "r")
    text = obj.read()
    # print(text)
    token = word_tokenize(text)
    word_size_before = len(token)
    # print(token)
    # removing the stopwords
    StopWords_Auditor = open(
        "StopWords\StopWords_Auditor.txt", "r", errors="ignore")
    auditor_text = StopWords_Auditor.read()
    auditor = word_tokenize(auditor_text)
    # print(auditor)
    filter_token_1 = []
    for word in token:
        if (word not in auditor):
            filter_token_1.append(word)
    # print(filter_token_1)

    StopWords_Currencies = open(
        "StopWords\StopWords_Currencies.txt", "r", errors="ignore")
    Currencies_text = StopWords_Currencies.read()
    Currencies = word_tokenize(Currencies_text)
    filter_token_2 = []
    for word in filter_token_1:
        if (word not in Currencies):
            filter_token_2.append(word)

    StopWords_DatesandNumbers = open(
        "StopWords\StopWords_DatesandNumbers.txt", "r")
    DatesandNumbers_text = StopWords_DatesandNumbers.read()
    DatesandNumbers = word_tokenize(DatesandNumbers_text)
    filter_token_3 = []
    for word in filter_token_2:
        if (word not in DatesandNumbers):
            filter_token_3.append(word)

    StopWords_Generic = open("StopWords\StopWords_Generic.txt", "r")
    Generic_text = StopWords_Generic.read()
    Generic = word_tokenize(Generic_text)
    filter_token_4 = []
    for word in filter_token_3:
        if (word not in Generic):
            filter_token_4.append(word)

    StopWords_GenericLong = open("StopWords\StopWords_GenericLong.txt", "r")
    GenericLong_text = StopWords_GenericLong.read()
    GenericLong = word_tokenize(GenericLong_text)
    filter_token_5 = []
    for word in filter_token_4:
        if (word not in GenericLong):
            filter_token_5.append(word)
    # print(filter_token_5)
    StopWords_Geographic = open("StopWords\StopWords_Geographic.txt", "r")
    Geographic_text = StopWords_Geographic.read()
    Geographic = word_tokenize(Geographic_text)
    filter_token_6 = []
    for word in filter_token_5:
        if (word not in Geographic):
            filter_token_6.append(word)
    # print(filter_token_6)

    StopWords_Names = open("StopWords\StopWords_Names.txt", "r")
    Names_text = StopWords_Names.read()
    Names = word_tokenize(Names_text)
    filter_token_7 = []
    for word in filter_token_6:
        if (word not in Names):
            filter_token_7.append(word)
    # print(filter_token_7)
    word_size_after = len(filter_token_7)
    # postive score
    positive_score = 0
    positive_words = open("MasterDictionary\\positive-words.txt", "r")
    pos_words_text = positive_words.read()
    pos_words = word_tokenize(pos_words_text)
    for word in filter_token_7:
        if (word in pos_words):
            positive_score += 1

    # print(positive_score)

    # Negative Score
    Negative_score = 0
    Negative_words = open("MasterDictionary\\negative-words.txt", "r")
    Neg_words_text = Negative_words.read()
    Neg_words = word_tokenize(Neg_words_text)
    for word in filter_token_7:
        if (word in Neg_words):
            Negative_score += 1

    # print(Negative_score)

    # Polarity Score
    Polarity_Score = (positive_score-Negative_score) /((positive_score+Negative_score)+0.000001)
    # print(Polarity_Score)
    # Subjectivity Score
    Subjectivity_Score = (positive_score + Negative_score) /((word_size_after) + 0.000001)
    # print(Subjectivity_Score)

    # sentences tokenizer
    from nltk.tokenize import sent_tokenize
    sentence_token = sent_tokenize(text)
    sentence_token_size = len(sentence_token)
    # print(sentence_token)

    # Analysis of Readability
    if(sentence_token_size!=0 and word_size_before!=0):
        Average_Sentence_Length = word_size_before/sentence_token_size
    # print(Average_Sentence_Length)

    # we need to find the no of complex world
    # we need to find syalbble count for each word
    complex_word_count = 0
    sum_of_characters = 0
    sum_of_syllables = 0
    for word in token:
        syllables, complex = count_syllables(word)
        sum_of_syllables+=syllables
        sum_of_characters += len(word)
        if (complex == 1):
            complex_word_count = complex_word_count+1
    if(word_size_after!=0):
        Percentage_Complex_words = complex_word_count/word_size_after

    if word_size_after !=0 :
        avg_syllable_count = sum_of_syllables/word_size_after
    # print(Percentage_Complex_words)

    # fog index
    fog_index = 0.4 * (sentence_token_size + Percentage_Complex_words)
    # print(fog_index)

    # average no of words per sentence
    if sentence_token_size !=0:
        avg_no_words_sentence = word_size_before/sentence_token_size
    # print(avg_no_words_sentence)

    word_count = 0

    for word in token:
        if (not word in stopwords_word and not word in punctuation_word):
            word_count += 1

    # Average Word Length
    if word_size_before !=0:
        Average_Word_Length = sum_of_characters/word_size_before
    # print(Average_Word_Length)

    # personal pronouns
    personal_pros = count_personal_pronouns(text)

    outputfile.loc[(outputfile['URL_ID'] == file), ['POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
                                                    'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
                                                    'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
                                                    'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
                                                    'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']] = positive_score, Negative_score, Polarity_Score, Subjectivity_Score, Average_Sentence_Length, Percentage_Complex_words, fog_index, avg_no_words_sentence, complex_word_count, word_count, avg_syllable_count, personal_pros, Average_Word_Length

    # print(outputfile[outputfile['URL_ID'] == file])

    # break

In [136]:
outputfile.to_excel("Output Data Structure.xlsx", 'sheet3')